In [21]:
# Imports
import pandas as pd
import numpy as np
import nltk 
import matplotlib.pyplot as plt
import csv
import random

In [22]:
reclustered_monthly_data = pd.read_csv("output_hpc/reclustered_monthly_data_final.csv", sep = "\t", encoding = "utf-8", quoting = csv.QUOTE_NONE )

In [23]:
data = reclustered_monthly_data.head(100)

In [24]:
n_rows = data.shape[0]

In [25]:
data["A"] = np.random.rand(n_rows)
data["F"] = np.random.rand(n_rows)
data["N"] = np.random.rand(n_rows)

/tmp/ipykernel_950/686247784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["A"] = np.random.rand(n_rows)
/tmp/ipykernel_950/686247784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["F"] = np.random.rand(n_rows)
/tmp/ipykernel_950/686247784.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [26]:
data_avg = data.groupby("user")[['A', 'F', 'N']].mean().reset_index()

In [27]:
max_val = data_avg.iloc[1][["A", "F", "N"]].max()
max_col = data_avg.iloc[1][["A", "F"]].idxmax()
print(max_val *-1)
print(max_val, max_col)

-0.4853049298901819
0.4853049298901819 A


In [28]:
def get_stance(row): 
    if row["N"] == 1.0: 
        return 0
    elif row["A"] > row["F"]:
        return row["A"]
    elif row["F"] > row["A"]:
        return row["F"]*-1
    else: 
        return random.choice([row["A"], row["F"]])

In [29]:
data_avg["stance"] = data_avg.apply(lambda x: get_stance(x), axis = 1)

#### Getting the stance for each user for each topic

In [30]:
def make_dataset(data):
    data = reclustered_monthly_data.head(100)
    n_rows = data.shape[0]
    data["A"] = np.random.rand(n_rows)
    data["F"] = np.random.rand(n_rows)
    data["N"] = np.random.rand(n_rows)
    data_avg = data.groupby("user")[['A', 'F', 'N']].mean().reset_index()
    data_avg["stance"] = data_avg.apply(lambda x: get_stance(x), axis = 1)
    return data_avg

In [31]:
data_1 = make_dataset(reclustered_monthly_data)
data_2 = make_dataset(reclustered_monthly_data)
data_3 = make_dataset(reclustered_monthly_data)

/tmp/ipykernel_950/2471956829.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["A"] = np.random.rand(n_rows)
/tmp/ipykernel_950/2471956829.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["F"] = np.random.rand(n_rows)
/tmp/ipykernel_950/2471956829.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [32]:
data1 = data_1.rename(columns={"stance": "stance_1"})
data2 = data_2.rename(columns={"stance": "stance_2"})
data3 = data_3.rename(columns={"stance": "stance_3"})

In [33]:
merged_df = pd.merge(pd.merge(data1, data2, on='user'), data3, on='user')

In [34]:
merged_df = merged_df[["user", "stance_1", "stance_2", "stance_3"]]
merged_df

,user,stance_1,stance_2,stance_3
0,AlbionPrince,-0.899503,-0.197501,0.467632
1,BenFoldsFourLoko,0.899054,-0.467889,0.930038
2,Betrashndie,-0.986026,0.959480,-0.280052
3,BloodWiz,0.648485,0.665346,-0.712709
4,Call_Me_Hurr1cane,0.494927,-0.352391,0.756536
...,...,...,...,...
91,thebigslapper,0.609985,0.620845,0.297749
92,throwrowrowawayyy,-0.758959,0.205901,0.903658
93,trogon,-0.575962,0.281986,0.832892
94,weeteacups,0.930559,-0.935738,0.537766


#### Dimensionality reduction - UMAP 

In [43]:
merged_df_umap = merged_df.copy()

#### Dimensionality reduction - PCA

In [35]:
merged_df_pca = merged_df.copy()

In [36]:
from sklearn.decomposition import PCA
import sklearn.preprocessing as pre

In [ ]:
# scaling the images
scaler = pre.StandardScaler()
merged_df_pca[["stance_1", "stance_2", "stance_3"]] = scaler.fit_transform(merged_df_pca[["stance_1", "stance_2", "stance_3"]])

In [39]:
pca = PCA(n_components=1)

In [40]:
merged_df_pca["pca_component"] = pca.fit_transform(merged_df_pca[["stance_1", "stance_2", "stance_3"]])

In [41]:
merged_df_pca

,user,stance_1,stance_2,stance_3,pca_component
0,AlbionPrince,-1.322767,-0.513050,0.528211,0.924612
1,BenFoldsFourLoko,1.239949,-0.913606,1.220091,1.317128
2,Betrashndie,-1.446052,1.200915,-0.590522,-1.009357
3,BloodWiz,0.882920,0.765182,-1.237890,-1.549382
4,Call_Me_Hurr1cane,0.664119,-0.742505,0.960486,1.098681
...,...,...,...,...,...
91,thebigslapper,0.828062,0.699258,0.274021,-0.379758
92,throwrowrowawayyy,-1.122510,0.084554,1.180619,0.985024
93,trogon,-0.861762,0.197267,1.074735,0.793859
94,weeteacups,1.284840,-1.606683,0.633148,1.331245


In [45]:
def apply_pca(merged_df_topics, colume_list): 
    # scaler
    scaler = pre.StandardScaler()
    merged_df_topics[colume_list] = scaler.fit_transform(merged_df_topics[colume_list])

    # pca 
    pca = PCA(n_components=1)
    merged_df_topics["pca_component"] = pca.fit_transform(merged_df_topics[["stance_1", "stance_2", "stance_3"]])
    return merged_df_topics

In [47]:
apply_pca(merged_df_pca, ["stance_1", "stance_2", "stance_3"])

,user,stance_1,stance_2,stance_3,pca_component
0,AlbionPrince,-1.322767,-0.513050,0.528211,0.924612
1,BenFoldsFourLoko,1.239949,-0.913606,1.220091,1.317128
2,Betrashndie,-1.446052,1.200915,-0.590522,-1.009357
3,BloodWiz,0.882920,0.765182,-1.237890,-1.549382
4,Call_Me_Hurr1cane,0.664119,-0.742505,0.960486,1.098681
...,...,...,...,...,...
91,thebigslapper,0.828062,0.699258,0.274021,-0.379758
92,throwrowrowawayyy,-1.122510,0.084554,1.180619,0.985024
93,trogon,-0.861762,0.197267,1.074735,0.793859
94,weeteacups,1.284840,-1.606683,0.633148,1.331245


#### Computing polarization 